# <center> HMM for POS tagging using Viterbi Algorithm </center>

In [1]:
import numpy as np
import pandas as pd

## Training Data

Training data is provided from https://www.clips.uantwerpen.be/conll2000/chunking/

In [2]:
Train_data = pd.read_csv('train.txt', delimiter=' ', names = ['Word', 'POS_Tag', 'Chunk_Tag'])

In [3]:
Train_data.head(50)

,Word,POS_Tag,Chunk_Tag
0,Confidence,NN,B-NP
1,in,IN,B-PP
2,the,DT,B-NP
3,pound,NN,I-NP
4,is,VBZ,B-VP
5,widely,RB,I-VP
6,expected,VBN,I-VP
7,to,TO,I-VP
8,take,VB,I-VP
9,another,DT,B-NP


In [4]:
Train_data.describe()

,Word,POS_Tag,Chunk_Tag
count,211727,211727,211727
unique,19122,44,22
top,",",NN,I-NP
freq,10770,30147,63307


In [5]:
Tag1 = Train_data.POS_Tag.unique()
print(Tag1.size)
Tag1

44


array(['NN', 'IN', 'DT', 'VBZ', 'RB', 'VBN', 'TO', 'VB', 'JJ', 'NNS',
       'NNP', ',', 'CC', 'POS', '.', 'VBP', 'VBG', 'PRP$', 'CD', '``',
       "''", 'VBD', 'EX', 'MD', '#', '(', '$', ')', 'NNPS', 'PRP', 'JJS',
       'WP', 'RBR', 'JJR', 'WDT', 'WRB', 'RBS', 'PDT', 'RP', ':', 'FW',
       'WP$', 'SYM', 'UH'], dtype=object)

In [6]:
Tag2 = Train_data.Chunk_Tag.unique()
print(Tag2.size)
Tag2

22


array(['B-NP', 'B-PP', 'I-NP', 'B-VP', 'I-VP', 'B-SBAR', 'O', 'B-ADJP',
       'B-ADVP', 'I-ADVP', 'I-ADJP', 'I-SBAR', 'I-PP', 'B-PRT', 'B-LST',
       'B-INTJ', 'I-INTJ', 'B-CONJP', 'I-CONJP', 'I-PRT', 'B-UCP',
       'I-UCP'], dtype=object)

In [7]:
Words = Train_data.Word
len(Words)

211727

In [8]:
Words_array = np.array(Words)
for i in range(500):
    print(Words_array[i], end=' ')

Confidence in the pound is widely expected to take another sharp dive if trade figures for September , due for release tomorrow , fail to show a substantial improvement from July and August 's near-record deficits . Chancellor of the Exchequer Nigel Lawson 's restated commitment to a firm monetary policy has helped to prevent a freefall in sterling over the past week . But analysts reckon underlying support for sterling has been eroded by the chancellor 's failure to announce any new policy measures in his Mansion House speech last Thursday . This has increased the risk of the government being forced to increase base rates to 16 % from their current 15 % level to defend the pound , economists and foreign exchange market analysts say . `` The risks for sterling of a bad trade figure are very heavily on the down side , '' said Chris Dillow , senior U.K. economist at Nomura Research Institute . `` If there is another bad trade number , there could be an awful lot of pressure , '' noted Si

## Pre-processing

As we can see that there are lot of special characters like ,''#()$. These need to be removed.

In [25]:
remove = [',', '``', "''", '#', '(', '$', ')', ':']
for item in remove:
    index_names = Train_data[Train_data.POS_Tag == item].index
    print(item, '->', len(index_names))
    Train_data.drop(index_names, inplace=True)

Train_data.describe()

, -> 0
`` -> 0
'' -> 0
# -> 0
( -> 0
$ -> 0
) -> 0
: -> 0


,Word,POS_Tag,Chunk_Tag
count,194545,194545,194545
unique,19103,36,22
top,the,NN,I-NP
freq,9219,30147,62109


In [10]:
# After removing special characters, the unique POS_Tags are:
Tag1 = Train_data.POS_Tag.unique()
print(Tag1.size)
Tag1

36


array(['NN', 'IN', 'DT', 'VBZ', 'RB', 'VBN', 'TO', 'VB', 'JJ', 'NNS',
       'NNP', 'CC', 'POS', '.', 'VBP', 'VBG', 'PRP$', 'CD', 'VBD', 'EX',
       'MD', 'NNPS', 'PRP', 'JJS', 'WP', 'RBR', 'JJR', 'WDT', 'WRB',
       'RBS', 'PDT', 'RP', 'FW', 'WP$', 'SYM', 'UH'], dtype=object)

In [11]:
Words = Train_data.Word
len(Words)

194545

In [12]:
Words_array = np.array(Words)
for i in range(500):
    print(Words_array[i], end=' ')

Confidence in the pound is widely expected to take another sharp dive if trade figures for September due for release tomorrow fail to show a substantial improvement from July and August 's near-record deficits . Chancellor of the Exchequer Nigel Lawson 's restated commitment to a firm monetary policy has helped to prevent a freefall in sterling over the past week . But analysts reckon underlying support for sterling has been eroded by the chancellor 's failure to announce any new policy measures in his Mansion House speech last Thursday . This has increased the risk of the government being forced to increase base rates to 16 % from their current 15 % level to defend the pound economists and foreign exchange market analysts say . The risks for sterling of a bad trade figure are very heavily on the down side said Chris Dillow senior U.K. economist at Nomura Research Institute . If there is another bad trade number there could be an awful lot of pressure noted Simon Briscoe U.K. economist

## Transition Probablity Matrix

In [13]:
POS_Tags = Train_data.POS_Tag.unique()
tags_matrix = np.zeros(shape=(len(POS_Tags),len(POS_Tags)))
tags_matrix.shape

(36, 36)

In [14]:
def t2_given_t1(t2, t1, train_bag = Train_data):
    tags = list(train_bag.POS_Tag)
    t1_tags_count = tags.count(t1)
    if t1_tags_count == 0:
        return 0
    
    t2_given_t1_list = [index for index in range(len(tags)-1) if tags[index] == t1 and tags[index+1] == t2]
    t2_given_t1_count = len(t2_given_t1_list)
    
    return(t2_given_t1_count/t1_tags_count)

In [15]:
for i, t1 in enumerate(list(POS_Tags)):
    for j, t2 in enumerate(list(POS_Tags)):
        tags_matrix[i, j] = t2_given_t1(t2, t1)
tags_matrix

array([[1.24091949e-01, 2.64901980e-01, 2.38166318e-02, ...,
        3.98049557e-04, 9.95123893e-05, 0.00000000e+00],
       [1.13029345e-01, 3.08820945e-02, 3.28061852e-01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [4.86664849e-01, 8.67193891e-03, 1.85437687e-03, ...,
        0.00000000e+00, 0.00000000e+00, 2.18161985e-04],
       ...,
       [3.71428571e-01, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 3.33333333e-01, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 6.66666667e-02, 1.33333333e-01, ...,
        0.00000000e+00, 0.00000000e+00, 6.66666667e-02]])

In [16]:
# convert the matrix to a data frame
tags_matrix_df = pd.DataFrame(tags_matrix, columns = list(POS_Tags), index=list(POS_Tags))
tags_matrix_df

,NN,IN,DT,VBZ,RB,VBN,TO,VB,JJ,NNS,...,JJR,WDT,WRB,RBS,PDT,RP,FW,WP$,SYM,UH
NN,0.124092,0.264902,0.023817,0.046373,0.023783,0.016221,0.043520,0.002455,0.015524,0.089064,...,0.001758,0.013102,0.002620,0.000365,0.000033,0.000100,0.000166,0.000398,0.000100,0.000000
IN,0.113029,0.030882,0.328062,0.001625,0.015727,0.006018,0.003470,0.000483,0.085486,0.062160,...,0.005403,0.003251,0.001757,0.001889,0.001230,0.000000,0.000264,0.000000,0.000000,0.000000
DT,0.486665,0.008672,0.001854,0.009217,0.011235,0.015817,0.000327,0.000764,0.201309,0.073521,...,0.006545,0.000218,0.000000,0.003054,0.000000,0.000000,0.000218,0.000000,0.000000,0.000218
VBZ,0.040448,0.100043,0.175559,0.003873,0.136403,0.145224,0.045611,0.003442,0.079819,0.018718,...,0.008176,0.000861,0.008391,0.001721,0.000215,0.001506,0.000000,0.000000,0.000000,0.000000
RB,0.012714,0.139549,0.070985,0.044347,0.063115,0.087180,0.026184,0.110338,0.107764,0.009535,...,0.017557,0.000908,0.004389,0.000605,0.000454,0.000000,0.000151,0.000000,0.000151,0.000000
VBN,0.094058,0.368885,0.067814,0.004199,0.045140,0.032543,0.105606,0.001470,0.048079,0.045979,...,0.005669,0.000420,0.003149,0.000210,0.000420,0.000840,0.000000,0.000000,0.000000,0.000000
TO,0.022633,0.004330,0.110411,0.000394,0.009841,0.000984,0.000197,0.584334,0.028538,0.025192,...,0.003543,0.000590,0.000590,0.000394,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
VB,0.066811,0.144092,0.214559,0.004986,0.047033,0.085591,0.047366,0.007146,0.084261,0.059000,...,0.013960,0.001496,0.005484,0.000831,0.000831,0.004487,0.000000,0.000000,0.000000,0.000000
JJ,0.461139,0.057929,0.008942,0.003133,0.007184,0.003592,0.030264,0.000764,0.077646,0.227589,...,0.000306,0.000611,0.000764,0.000153,0.000076,0.000229,0.000076,0.000076,0.000000,0.000000
NNS,0.026067,0.245392,0.033923,0.014685,0.042661,0.026140,0.042074,0.002350,0.025846,0.020706,...,0.001689,0.017035,0.002643,0.000661,0.000073,0.000000,0.000073,0.000661,0.000000,0.000000


## Emission Probablity

In [17]:
# compute emission probability for a given word for a given tag
def word_given_tag(word, tag, train_bag = Train_data):
#     taglist = [pair for pair in train_bag if pair[1]
    taglist = train_bag[train_bag.POS_Tag == tag]
    tag_count = len(taglist)    
#     w_in_tag = [pair[0] for pair in taglist if pair[0]==word]    
    w_in_tag = taglist[taglist.Word == word]
    word_count_given_tag = len(w_in_tag)
    
    return (word_count_given_tag/tag_count)

## Viterbi Algorithm

In [18]:
# Occurance of Tags in the Training Data
len_of_data = len(Train_data.POS_Tag)
tag_prob = {}
tags = Train_data.POS_Tag.unique()
for tag in tags:
    tag_count = len(Train_data[Train_data.POS_Tag == tag])
    tag_prob[tag] = tag_count/len_of_data
tag_prob

{'NN': 0.1549615770130304,
 'IN': 0.11701148834459893,
 'DT': 0.09424554730268062,
 'VBZ': 0.02389164460664628,
 'RB': 0.03396129430208949,
 'VBN': 0.024482767483101596,
 'TO': 0.026117350741473696,
 'VB': 0.03092857693592742,
 'JJ': 0.06725950294276388,
 'NNS': 0.07000436916908684,
 'NNP': 0.10220771543858748,
 'CC': 0.027613148628851936,
 'POS': 0.009093011899560513,
 '.': 0.045372535917139994,
 'VBP': 0.014742090518903081,
 'VBG': 0.016818730884885245,
 'PRP$': 0.009668714179238737,
 'CD': 0.0427407540671824,
 'VBD': 0.03467064175383587,
 'EX': 0.0010588809786938754,
 'MD': 0.011138811071988487,
 'NNPS': 0.0021588835487933384,
 'PRP': 0.01963555989616798,
 'JJS': 0.0019224343982112107,
 'WP': 0.0027191652316944665,
 'RBR': 0.0016500038551491942,
 'JJR': 0.004384589683620756,
 'WDT': 0.004908889974041995,
 'WRB': 0.002457015086483847,
 'RBS': 0.0009817779948083992,
 'PDT': 0.00028271094091341335,
 'RP': 0.0004266365108329692,
 'FW': 0.00019532755917654014,
 'WP$': 0.00017990696239944

In [19]:
def Viterbi(words, train_bag = Train_data):
    state = []
    
    Tags = train_bag.POS_Tag.unique() # Unique Tags
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = []
        p_transition = []
        for tag in Tags:
            if key == 0:
                transition_p = tags_matrix_df.loc['.', tag]
            else:
                transition_p = tags_matrix_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)
            state_probability = emission_p * transition_p
            unknown_word_probability = tag_prob[tag] * transition_p
            p.append(state_probability)
            p_transition.append(unknown_word_probability)
            
        pmax = max(p)
        # getting state for which probability is maximum
        if pmax == 0:
            state_max = Tags[p_transition.index(max(p_transition))]
        else:
            state_max = Tags[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))

## Test Data

Testing data is provided from same https://www.clips.uantwerpen.be/conll2000/chunking/

In [27]:
Test_data = pd.read_csv('test.txt', delimiter=' ', names = ['Word', 'POS_Tag', 'Chunk_Tag'])
Test_data.head(50)

,Word,POS_Tag,Chunk_Tag
0,Rockwell,NNP,B-NP
1,International,NNP,I-NP
2,Corp.,NNP,I-NP
3,'s,POS,B-NP
4,Tulsa,NNP,I-NP
5,unit,NN,I-NP
6,said,VBD,B-VP
7,it,PRP,B-NP
8,signed,VBD,B-VP
9,a,DT,B-NP


In [28]:
Test_data.describe()

,Word,POS_Tag,Chunk_Tag
count,47377,47377,47377
unique,8118,43,19
top,",",NN,I-NP
freq,2390,6642,14376


In [29]:
print(len(Test_data.POS_Tag.unique()))
Test_data.POS_Tag.unique()

43


array(['NNP', 'POS', 'NN', 'VBD', 'PRP', 'DT', 'JJ', 'VBG', 'PRP$', 'IN',
       'TO', 'VB', 'NNS', 'CD', '.', 'VBZ', 'VBP', ',', 'VBN', 'CC', 'RB',
       'WDT', 'WP', '$', 'RBR', 'JJR', 'NNPS', 'MD', 'WRB', ':', 'JJS',
       '``', 'EX', "''", '(', ')', 'WP$', 'RP', 'UH', 'RBS', 'FW', 'PDT',
       '#'], dtype=object)

### Pre-processing

As we can see that there are lot of special characters like ,''#()$. These need to be removed.

In [30]:
remove = [',', '``', "''", '#', '(', '$', ')', ':']
for item in remove:
    index_names = Test_data[Test_data.POS_Tag == item].index
    print(item, '->', len(index_names))
    Test_data.drop(index_names, inplace=True)

Test_data.describe()

, -> 2390
`` -> 323
'' -> 316
# -> 11
( -> 77
$ -> 384
) -> 77
: -> 238


,Word,POS_Tag,Chunk_Tag
count,43561,43561,43561
unique,8102,35,19
top,the,NN,I-NP
freq,2059,6642,14078


### Testing

In [32]:
test_tagged_words = np.array(Test_data.Word)
print(len(test_tagged_words))
# test_tagged_words[:500]

43561


In [33]:
import time
start = time.time()
tagged_seq = Viterbi(test_tagged_words[:500])
end = time.time()
difference = end-start

print("Time taken in seconds: ", difference)

Time taken in seconds:  221.15766787528992


In [34]:
# accuracy
check = [i for i, j in zip(tagged_seq, np.array(Test_data.POS_Tag.iloc[:500])) if i[1] == j] 

accuracy = len(check)/len(tagged_seq)
print('Vanilla Viterbi Algorithm Accuracy: ',accuracy*100)

Vanilla Viterbi Algorithm Accuracy:  92.80000000000001


## Check your own sentence

In [35]:
string = input("Write a sentence: ").split(' ')

Write a sentence: Nothing is so painful to the human mind as a great and sudden change


In [36]:
# Tags for ecah word according to the Traing data
flag = 1
dictionary = dict()
for i in string:
    index_of_word = Train_data[Train_data.Word == i].index
    tag1 = Train_data.loc[index_of_word].POS_Tag.unique()
    if flag:
        data_frame = pd.DataFrame(data = [tag1])
        flag = 0
    else:
        data_frame = data_frame.append([tag1])
s = pd.Series(string)
data_frame.set_index([s], inplace=True)
data_frame

,0,1,2
Nothing,NN,NaN,NaN
is,VBZ,NNS,NaN
so,RB,IN,NaN
painful,JJ,NaN,NaN
to,TO,NaN,NaN
the,DT,IN,NaN
human,JJ,NaN,NaN
mind,NN,VB,NaN
as,RB,IN,NaN
a,DT,NaN,NaN


In [37]:
vit = Viterbi(string)

In [38]:
pd.DataFrame(data = vit)

,0,1
0,Nothing,NN
1,is,VBZ
2,so,RB
3,painful,JJ
4,to,TO
5,the,DT
6,human,JJ
7,mind,NN
8,as,IN
9,a,DT
